In [1]:
import numpy as np
from utils.generator import generate_positive_definite_matrix, make_quadratic_function
from itertools import product
from scipy.optimize import minimize
from tqdm import tqdm

In [2]:
n = 20

In [3]:
Q = generate_positive_definite_matrix(n)
c = np.random.normal(size=n)
mu0 = 1.5 / np.min(np.linalg.eigvalsh(Q))

In [4]:
def brute_force(Q, c):
    n = len(c)
    obj_fn = make_quadratic_function(Q, c)
    all_possible_x = np.array(list(product([0, 1], repeat=n)))
    x_star = all_possible_x[np.argmin(np.array(list(map(obj_fn, all_possible_x))))]
    return x_star

In [5]:
def binary_admm(obj_fn, x0, y0, mu0, n_iter=100, gamma=0.9):
    n = len(x0)
    x = z = x0
    y = y0
    mu = mu0
    bnds = [(0,1)] * n
    for _ in range(n_iter):
        lx = lambda x: obj_fn(x) + np.dot(y, x - z) + 1/(2*mu) * (np.linalg.norm(x - z) ** 2)
        res = minimize(lx, x, bounds=bnds)
        x = res.x
        z = np.where(x ** 2 / (2 * mu) < (x - 1) ** 2 / (2 * mu) - y, 0, 1)
        y += (x - z) / mu
        mu *= gamma
    return x, y

In [6]:
correct = 0
n_iter = 100
for i in tqdm(range(n_iter)):
    Q = generate_positive_definite_matrix(n)
    c = np.random.normal(size=n)
    k = np.min(np.linalg.eigvalsh(Q))
    obj_fn = make_quadratic_function(Q, c)
    mu = 1/k * 1.5
    xhat, yhat = binary_admm(obj_fn, np.ones(n) * 0.5, np.zeros(n), mu, 100, gamma=0.9)
    xhat = np.where(xhat > 0.5, 1, 0)
    xstar = brute_force(Q, c)
    if np.sum(xhat ^ xstar):
        print(mu, np.sum(xhat ^ xstar))
    correct += 1 - np.sum(xhat ^ xstar) / n

print(correct / n_iter)

  6%|▌         | 6/100 [00:44<11:24,  7.28s/it]

100.67908587252519 1


  7%|▋         | 7/100 [00:51<11:19,  7.31s/it]

121.75210034671841 2


  9%|▉         | 9/100 [01:06<11:09,  7.36s/it]

...

123.0963304030446 1


 95%|█████████▌| 95/100 [11:47<00:37,  7.43s/it]

145.62334392683636 2


100%|██████████| 100/100 [12:23<00:00,  7.44s/it]

0.9785000000000005
